# 1. 패키지 설치

In [2]:
# %pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

# 2. Knowledge Base 구성을 위한 데이터 생성

- Chroma를 활용한 [3.2 LangChain과 Chroma를 활용한 RAG 구성](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.2%20LangChain%EA%B3%BC%20Chroma%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20RAG%20%EA%B5%AC%EC%84%B1.ipynb)과 동일함
- Vector Database만 [Pinecone](https://www.pinecone.io/)으로 변경

In [25]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [26]:
document_list[52]

Document(page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 항 각 호 외의 부분 단서에 따라 해당 과세기간의 총수입금액에 더한 금액에 해당하는 금액을 종합소득 산출세액에서 공제한다. <개정 2009. 12. 31.>\n\n② 제1항에 따른 공제를 “배당세액공제”라 한다.<개정 2009. 12. 31.>\n\n③ 삭제<2003. 12. 30.>\n\n④ 제1항을 적용할 때 배당세액공제의 대상이 되는 배당소득금액은 제14조제2항의 종합소득과세표준에 포함된 배당소득금액으로서 이자소득등의 종합과세기준금액을 초과하는 것으로 한다.<개정 2009. 12. 31.>\n\n⑤ 삭제<2006. 12. 30.>\n\n⑥ 배당세액공제액의 계산 등에 필요한 사항은 대통령령으로 정한다.<개정 2009. 12. 31.>\n\n[제목개정 2009. 12. 31.]\n\n\n\n제56조의2(기장세액공제) ① 제160조제3항에 따른 간편장부대상자가 제70조 또는 제7

In [27]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [28]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [29]:
query = '연봉 5천만원인 거주자의 종합소득세는?'

# 3. 답변 생성을 위한 Retrieval

- `RetrievalQA`에 전달하기 위해 `retriever` 생성
- `search_kwargs` 의 `k` 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- `.invoke()` 를 호출해서 어떤 문서를 가져오는지 확인 가능

In [30]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [31]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

[]

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [33]:
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [36]:
# LangChain 권장사항에 따라 강의 코드와 다르게 `.invoke()`를 사용합니다
ai_message = qa_chain.invoke({"query": query})

In [35]:
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉 5천만원인 거주자의 종합소득세를 계산하려면, 먼저 총급여액에서 근로소득공제를 적용한 후 과세표준을 산출하고, 기본세율을 적용해야 합니다. 그러나 구체적인 계산을 위해 세율과 공제액 등을 고려해야 하므로, 정확한 종합소득세 금액을 제공할 수 없습니다. 세무 전문가와 상담하여 정확한 금액을 확인하시기 바랍니다.'}